# Aprendizagem de Máquina I

## Hugo Tremonte de Carvalho

#### hugo@dme.ufrj.br

"*In this challenge, we invite Kagglers to help us identify which customers will make a specific transaction in the future, irrespective of the amount of money transacted. The data provided for this competition has the same structure as the real data we have available to solve this problem.*"

https://www.kaggle.com/c/santander-customer-transaction-prediction/overview

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import RocCurveDisplay, ConfusionMatrixDisplay, confusion_matrix, roc_curve

import numpy as np

import scipy.stats as stats

In [ ]:
class ConfusionMatrixMetrics:
    def __init__(self, matrix):
        """
        Inicializa a classe com uma matriz de confusão 2x2.
        A matriz deve estar no formato:
        [[VN, FP],
         [FN, VP]]
        onde:
        - VP: Verdadeiros Positivos
        - FP: Falsos Positivos
        - FN: Falsos Negativos
        - VN: Verdadeiros Negativos
        """
        self.VN = matrix[0][0]
        self.FP = matrix[0][1]
        self.FN = matrix[1][0]
        self.VP = matrix[1][1]

        self.N = matrix[0][0] + matrix[0][1]
        self.P = matrix[1][0] + matrix[1][1]

        self.Pop = self.N + self.P
    
    def prev(self):
        """Calcula a prevalência da classe positiva"""
        try:
            return (self.P)/(self.Pop)
        except ZeroDivisionError:
            return 0.0
    
    def acc(self):
        """Calcula a acurária"""
        try:
            return (self.VN + self.VP)/self.Pop
        except ZeroDivisionError:
            return 0.0

    def FPR(self):
        """Calcula a FPR"""
        try:
            return self.FP/self.N
        except ZeroDivisionError:
            return 0.0

    def TNR(self):
        """Calcula a TNR"""
        try:
            return self.VN/self.N
        except ZeroDivisionError:
            return 0.0

    def TPR(self):
        """Calcula a TPR"""
        try:
            return self.VP/self.P
        except ZeroDivisionError:
            return 0.0
    
    def FNR(self):
        """Calcula a FNR"""
        try:
            return self.FN/self.P
        except ZeroDivisionError:
            return 0.0
        
    def FOR(self):
        """Calcula a FOR"""
        try:
            return self.FN/(self.VN + self.FN)
        except ZeroDivisionError:
            return 0.0

    def PPV(self):
        """Calcula o PPV"""
        try:
            return self.VP/(self.FP + self.VP)
        except ZeroDivisionError:
            return 0.0

    def NPV(self):
        """Calcula a NPV"""
        try:
            return self.VN/(self.VN + self.FN)
        except ZeroDivisionError:
            return 0.0

    def FDR(self):
        """Calcula a FDR"""
        try:
            return self.FP/(self.FP + self.VP)
        except ZeroDivisionError:
            return 0.0

    def F1(self):
        """Calcula a F1"""
        try:
            return 2/(1/self.PPV() + 1/self.TPR())
        except ZeroDivisionError:
            return np.NaN
    
    def print(self):
        """Mostra todas as métricas calculadas acima"""
        print('Prevalência:', np.round(self.prev(), 3))
        print('Acurácia:', np.round(self.acc(), 3))

        print('\n')

        print('Taxa de falsos positivos:', np.round(self.FPR(), 3))
        print('Taxa de verdadeiros negativos (Especificidade):',np.round(self.TNR(), 3))
        print('Taxa de verdadeiros positivos (Recall):', np.round(self.TPR(), 3))
        print('Taxa de falsos negativos:', np.round(self.FNR(), 3))

        print('\n')

        print('False omission rate:', np.round(self.FOR(), 3))
        print('Valor preditivo positivo (Precisão):', np.round(self.PPV(), 3))
        print('Valor preditivo negativo:', np.round(self.NPV(), 3))
        print('False discovery rate:', np.round(self.FDR(), 3))

        print('\n')

        print('F1 Score:', np.round(self.F1(), 3))

a) Faça uma análise exploratória na base de dados, verificando se as hipóteses de nossos classificadores gaussianos se aplicam. Aproveite para fazer uma limpeza na base.

In [ ]:
# VERSÃO REDUZIDA DA BASE DE DADOS, POR CONTA DE LIMITAÇÕES DE ESPAÇO DO GITHUB

banco = pd.read_csv('https://raw.githubusercontent.com/HugoCarvalhoUFRJ/ap-maq/refs/heads/main/materiais-didaticos/bank_train_redux.csv')

In [ ]:
banco.head()

In [ ]:
banco = banco.replace(to_replace = ';', value = '', regex = True)
banco.head()

In [ ]:
banco = banco.rename(columns={'var_199;;;;;;;':'var_199'})
banco.head()

In [ ]:
print(list(banco.columns))

In [ ]:
banco.dtypes

In [ ]:
banco['var_199']

In [ ]:
banco['var_199'] = pd.to_numeric(banco['var_199'])

In [ ]:
banco.dtypes

In [ ]:
banco = banco.drop(columns = 'ID_code')
banco.head()

In [ ]:
banco.describe()

In [ ]:
banco['target'].value_counts()

In [ ]:
X = banco.iloc[:, 1:]
X.head()

In [ ]:
y = banco['target']
y.head()

In [ ]:
# https://en.wikipedia.org/wiki/Kernel_density_estimation

X[y == 0].plot(kind = 'kde', ind = 10, legend = False, figsize = (15, 5))
plt.title('Estimação da densidade dos atributos para a classe negativa')
plt.xlabel('Atributo')
plt.ylabel('Densidade')
plt.show()

X[y == 1].iloc[:, 2:].plot(kind = 'kde', ind = 10, legend = False, figsize = (15, 5))
plt.title('Estimação da densidade dos atributos para a classe positiva')
plt.xlabel('Atributo')
plt.ylabel('Densidade')
plt.show()

In [ ]:
dominio = np.linspace(-5, 5, 100)

pd.DataFrame(StandardScaler().fit_transform(X))[y == 0].plot(kind = 'kde', ind = 11, legend = False, figsize = (15, 5))
plt.plot(dominio, stats.norm.pdf(dominio, 0, 1), '--k')
plt.title('Estimação da densidade dos atributos para a classe negativa')
plt.xlabel('Atributo')
plt.ylabel('Densidade')
plt.show()

pd.DataFrame(StandardScaler().fit_transform(X))[y == 1].iloc[:, 2:].plot(kind = 'kde', ind = 11, legend = False, figsize = (15, 5))
plt.plot(dominio, stats.norm.pdf(dominio, 0, 1), '--k')
plt.title('Estimação da densidade dos atributos para a classe positiva')
plt.xlabel('Atributo')
plt.ylabel('Densidade')
plt.show()

In [ ]:
X[y == 0].corr()

In [ ]:
plt.figure(figsize = (6, 6))
plt.imshow(X[y == 0].corr())
plt.colorbar()
plt.title('Estimação da correlação entre os atributos \n para a classe negativa')
plt.show()

plt.figure(figsize = (6, 6))
plt.imshow(X[y == 1].corr())
plt.colorbar()
plt.title('Estimação da correlação entre os atributos \n para a classe positiva')
plt.show()

b) Divida a base em treino e teste, e valide o desempenho dos classificadores gaussianos. Aproveite para ler a documentação desses classificadores:

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html

https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html

c) Melhore o desempenho dos classificadores treinados no item anterior, adequando o ponto de corte para classificar uma instância como `1` ou `0`. Calcule as métricas nesse novo cenário.